# Therme source volumique d'échange avec une température extérieure

In [ ]:
from trustutils import run  

run.introduction("A. Gerschenfeld, Y. Gorsse")

### Description

Cette fiche teste le terme source Terme_Puissance_Thermique_Echange_Impose en VEF et PolyMAC. Ce terme est utilisé en pratique pour calculer un échangeur : la partir primaire est calculée en CFD poreuse, et le secondaire est imposé dans le jdd ou bien calculé en couplage. Le test consiste en une cavité au repos, à 30degres initialement. Le terme source d'échange est appliqué avec une température externe à 10degres. On s'attend à voir la puissance échangée tendre vers zéro avec le temps.

In [ ]:
run.TRUST_parameters()

In [ ]:
from trustutils import run

run.reset()
power = -1000
Text = 10
regul_str = f"PID_controler_on_targer_power {{ target_power {power} Kp 1e-6 Ki 1e-1 Kd 0 }}"
dic_dis = {"VEFPreP1B" : "trianguler_h dom_fluide", "PolyMAC" : "", "PolyMAC_P0" : "", "VDF" : ""}
for dis, tri in dic_dis.items():
    run.addCaseFromTemplate("base.data", directory=dis, d={"dis": dis, "tri" : tri, "text" : Text})
    run.addCaseFromTemplate("regul.data", directory=dis, d={"dis": dis, "tri" : tri, "regul" : "", "text" : Text})
    if dis != "VEFPreP1B": run.addCaseFromTemplate("regul.data", directory=f"{dis}/regul", d={"dis": dis, "tri" : tri, "regul" : regul_str, "text" : Text})
run.printCases()
run.runCases()
run.tablePerf()

## Resultats 
### Puissance échangée via le terme source

In [ ]:
from trustutils import plot
import numpy as np

d_m = {"VEFPreP1B" : "-|", "PolyMAC" : "-x", "PolyMAC_P0" : "-", "VDF" : "-*"}

a = plot.Graph("Power")
b = plot.Graph("Text")
for d, m in d_m.items():
    x = plot.loadText(f"{d}/base_pb_Terme_Puissance_Thermique_Echange_Impose.out")
    a.add(x[0],x[1],marker=m,label=f"{d} closed cavity", color="blue")
    if d != "VEFPreP1B": b.add(x[0],[Text + DT for DT in x[2]],marker=m,label=f"{d} closed cavity", color="blue")
    x = plot.loadText(f"{d}/regul_pb_Terme_Puissance_Thermique_Echange_Impose.out")
    a.add(x[0],x[1],marker=m,label=f"{d} open cavity", color="green")
    if d != "VEFPreP1B": b.add(x[0],[Text + DT for DT in x[2]],marker=m,label=f"{d} open cavity", color="green")
    if d != "VEFPreP1B":
        x = plot.loadText(f"{d}/regul/regul_pb_Terme_Puissance_Thermique_Echange_Impose.out")
        a.add(x[0],x[1],marker=m,label=f"{d} open cavity with regulation", color="red")
        b.add(x[0],[Text + DT for DT in x[2]],marker=m,label=f"{d} open cavity with regulation", color="red")

a.label("Time [s]","Puissance [W]")
b.label("Time [s]","Text")
a.visu()